<a href="https://colab.research.google.com/github/shubhamcodez/mean-reversion-trading-strategy/blob/main/Mean_reversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mean Reversion based trading strategy

In [3]:
# Import necessary libraries
!pip install plotly
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
# Override yfinance API to use Pandas DataReader
yf.pdr_override()

In [4]:
# Define the stock symbol and time frame for mean calculation
symbol = "MSFT"
mean_frame = 10

# Define the start and end dates for fetching the data
start_date = '1999-06-23'
end_date = '2023-10-12'

# Fetch stock data using yfinance
stock_data = pd.DataFrame(yf.download(symbol, start=start_date, end=end_date)['Close'])

# Define starting amount for investment
starting_amount = 1000

[*********************100%%**********************]  1 of 1 completed


In [5]:
#system
def calculate_transaction_costs(total_trades, fixed_cost_per_trade):
    total_transaction_cost = total_trades * fixed_cost_per_trade
    return total_transaction_cost

In [6]:
def buy_and_hold(stock_data, starting_amount):
    stock_data['returns'] = stock_data['Close'].pct_change().fillna(0)  # Calculate daily returns and fill NaN values with 0
    stock_data['BuyAndHoldAmount'] = (1 + stock_data['returns']).cumprod() * starting_amount
    final_amount = stock_data['BuyAndHoldAmount']  # Get the final amount at the last row
    return final_amount

In [7]:
# Define the function for mean reversion strategy
def mean_reversion_strategy(stock_data, mean_frame=200, buy_threshold=10, sell_threshold=90):
    # Calculate mean using rolling window
    mean_data = stock_data['Close'].rolling(window=mean_frame).mean()

    # Calculate daily returns and z-scores for mean reversion
    stock_data['returns'] = stock_data['Close'].pct_change()
    stock_data['z_score'] = (stock_data['Close'] - mean_data) / stock_data['Close'].rolling(window=mean_frame).std()

    # Define entry and exit thresholds using user-defined percentiles
    buy = np.percentile(stock_data['z_score'].dropna(), buy_threshold)
    sell = np.percentile(stock_data['z_score'].dropna(), sell_threshold)
    stock_data['Signal'] = np.where(stock_data['z_score'] > sell, -1, np.where(stock_data['z_score'] < buy, 1, 0))

    # Prepare data for backtesting
    stock_data['Signal'] = stock_data['Signal'].shift(1)  # Shift signals by one day for accurate backtesting
    stock_data['StrategyReturn'] = stock_data['Signal'] * stock_data['returns']
    stock_data['StrategyReturn'] = stock_data['StrategyReturn'].fillna(0)

    stock_data['MeanReversionAmount'] = (1 + stock_data['StrategyReturn']).cumprod() * starting_amount

    # Extracting trade signals
    buy_signals = stock_data[stock_data['Signal'] == 1]
    sell_signals = stock_data[stock_data['Signal'] == -1]

    # Return necessary data for plotting
    return stock_data.index, stock_data['MeanReversionAmount'], buy_signals, sell_signals

In [8]:
buy_and_hold_amount = buy_and_hold(stock_data, starting_amount)
index, mean_reversion_amount, buy_signals, sell_signals = mean_reversion_strategy(stock_data, mean_frame)

In [9]:
import plotly.graph_objects as go

# Create interactive plot using plotly
fig = go.Figure()

# Add Buy & Hold trace
fig.add_trace(go.Scatter(x=index, y=buy_and_hold_amount, mode='lines', name='Buy & Hold Amount'))

# Add Mean Reversion trace
fig.add_trace(go.Scatter(x=index, y=mean_reversion_amount, mode='lines', name='Mean Reversion Amount'))

# Add buy signals as smaller and transparent green dots
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['MeanReversionAmount'], mode='markers',
                         marker=dict(color='green', size=6, opacity=0.7), name='Buy Signal'))

# Add sell signals as smaller and transparent red dots
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['MeanReversionAmount'], mode='markers',
                         marker=dict(color='red', size=6, opacity=0.7), name='Sell Signal'))

# Update layout for better interactivity
fig.update_layout(
    title='Buy & Hold vs Mean Reversion Strategy with Trade Signals',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Amount'),
    legend=dict(x=0, y=1, traceorder='normal'),
    margin=dict(l=0, r=0, t=30, b=0)
)

# Show the interactive plot
fig.show()

In [10]:
# Print final amounts
print("Final Buy & Hold Amount:", buy_and_hold_amount.iloc[-1])
print("Final Mean Reversion Amount:", mean_reversion_amount.iloc[-1])

Final Buy & Hold Amount: 7730.697986691519
Final Mean Reversion Amount: 5576.215950571873


In [11]:
# Calculate CAGR for Buy & Hold strategy
buy_and_hold_cagr = 100*((buy_and_hold_amount.iloc[-1] / starting_amount) ** (1 / (len(stock_data) / 252)) - 1)

# Calculate CAGR for Mean Reversion strategy
mean_reversion_cagr = 100*((mean_reversion_amount.iloc[-1] / starting_amount) ** (1 / (len(stock_data) / 252)) - 1)

print("Buy & Hold CAGR:", buy_and_hold_cagr , "%")
print("Mean Reversion CAGR:", mean_reversion_cagr , "%")

Buy & Hold CAGR: 8.792168017422908 %
Mean Reversion CAGR: 7.33756258116447 %
